<a href="https://colab.research.google.com/github/Cours-EDUlib/FAS-ISDS/blob/main/module-6/Applications/6_2_syntaxe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Les statistiques et la science des données appliquées
##Application 2: identifier et gérer des données extrêmes
###Syntaxe de l'exemple présenté

In [ ]:
#Téléchargement des packages nécessaires
library (tidyverse)
install.packages("rsq")
require(rsq)
library(ggplot2)
install.packages("descr")
require(descr)

Ceci est la suite du premier exemple sur la régression linéaire. Nous utilisons donc la même base de données concernant les États américains. On s'intéresse au taux de chômage et au taux de décrochage scolaire au sein de chacun des états. 

In [ ]:
path_to_csv = 'https://github.com/Cours-EDUlib/FAS-ISDS/blob/main/module-7/Applications/7-1-2_donnees.csv?raw=true' #Téléchargement de la base de données depuis github

bd <- read_csv(path_to_csv) #Enregistrement de la base de données dans l'ojet "bd"
bd #Lecture des données

Voici les variables à l'étude dans cet exemple:
- jbs170: taux de chômage par état en pourcentage
- scs134: taux de décrochage scolaire par état en pourcentage

Commençons par produire quelques statistiques descriptives de base pour se familiariser avec les variables à l’étude:
  - La moyenne
  - La médiane
  - Le minimum
  - Le maximum
  - l'écart-type

In [ ]:
#Statistiques descriptives

summary(bd$scs134)
sd(bd$scs134)

summary(bd$jbs170)
sd(bd$jbs170)

Interprétation:

Le taux de décrochage scolaire varie de 4,3% à 19,1%, avec une moyenne de 10,28% et un écart-type de 2,75% autour de la moyenne. Du côté du taux de chômage, il varie de 2,7% et 15,2%, avec une moyenne de 5,02% et un écart-type de 1,84%.

Par la suite, nous produisons des histogrammes, pour vérifier si les variables respectent le postulat de normalité de distribution. 

In [ ]:
#Histogramme pour vérification du postulat de normalité

ggplot(bd, aes(x=scs134)) +
  geom_histogram(binwidth=1, fill="#69b3a2", color="#e9ecef", alpha=0.9) +
    ggtitle("Distribution: taux de décrochage scolaire") +
    theme(
      plot.title = element_text(size=20)
    )

ggplot(bd, aes(x=jbs170)) +
  geom_histogram(binwidth=1, fill="#69b3a2", color="#e9ecef", alpha=0.9) +
    ggtitle("Distribution: taux de chômage") +
    theme(
      plot.title = element_text(size=20)
    )

Interprétation:

Pour le taux de chômage, la distribution n’est pas tout à fait normale, surtout en raison d’un état qui semble avoir un taux de chômage beaucoup plus élevé que l'ensemble des autres (autour de 15). En faisant abstraction de cet état, le reste de la distribution semble normale, avec des résultats qui varient entre 2,5 et 7,5%. On considère donc que le postulat est respecté.

Pour le taux de décrochage scolaire, l'histogramme montre une distribution très proche d’une distribution normale: le postulat est respecté.

Les deux variables sont distribuées assez normalement, donc on peut les utiliser pour faire une analyse de régression linéaire. 

Enfin, pour avoir un aperçu de la relation entre les variables, nous produisons un graphique de dispersion, ou nuage de points.

On se sert également du nuage de points pour confirmer l'existence d'une donnée extrême et l'indentifier plus précisément.

In [ ]:
#Nuage de points

nuage <- ggplot(bd, aes(x=scs134, y=jbs170)) + 
            geom_point() +
            geom_smooth(method=lm , color="black", se=FALSE) 

nuage + labs(x = "Taux de décrochage scolaire", y = "Taux de chômage")

Interprétation:

Le nuage de point représente la relation entre le taux de décrochage scolaire, la variable indépendante placée sur l’axe des X, et le taux de chômage, la variable dépendante placée sur l’axe des Y. Chaque point représente un état. On aperçoit  une tendance générale positive, où le taux de chômage augmente avec le taux de décrochage scolaire. Donc plus le taux de décrochage est élevé, plus le taux de chômage l’est également et vice-versa. Toutefois, quelques points semblent s’éloigner du nuage de tendance générale, dont une donnée au sommet du graphique qui semble correspondre à une donnée extrême.

Cette donnée extrême est tellement éloignée des autres qu'elle a assurément un impact important sur les résulats. Il faut donc la retirer; pour cela, il faut d'abord identifier à quel état (observation) correpsond ce point qu'on qualifie d'extrême. 

Tout d'abord, on peut identifier les points directement dans le nuage de points pour savoir à quelle observation correspond notre point éloigné de la moyenne.

In [ ]:
#Identifier un point dans un graphique
nuage2 <- ggplot(bd, aes(x=scs134, y=jbs170)) + 
            geom_point() +
            geom_smooth(method=lm , color="black", se=FALSE) +
            geom_text(
               label=rownames(bd), 
               nudge_x = 0.25, nudge_y = 0.25, 
               check_overlap = T
            )

nuage2

On peut aussi identifier les données extrêmes à l'étape suivante, soit la réalisation de la régression linéaire en elle-même. Les statistiques de résidus standardisés produites avec la fonction rstandard nous permettent de déceler l'existence d'une donnée extrême dépassant l'intervalle de -3 à 3 écarts-types de la moyenne. 

*Ne pas se fier aux "deviance residuals" produits automatiquement dans la régression linéaire: ceux-ci ne sont pas standardisés en fonction des écarts-types

In [ ]:
#Faire la régression linéaire

reglineaire <- glm(jbs170 ~ scs134, data = bd, family="gaussian")
summary(rstandard(reglineaire))
summary(reglineaire)
rsq(reglineaire)

Interprétation:

Avant le retrait des données extrêmes, la régression linéaire démontre l'existence d'une relation positive, de sorte que le taux de chômage augmente d’environ 0,21% pour chaque augmentation d’une unité du taux de décrochage scolaire. La relation est faible, mais positive et significative (p < 0.05). Selon le R-carré, environ 10% de la variance du taux de chômage est expliqué par le taux de décrochage scolaire.


Ensuite il faut trouver quelle est l'observation qui s'éloigne autant de la moyenne. Une première méthode, applicable dans de petites bases de données, est de sortir les statistiques de résidus de chaque observation et de trouver celle(s) qui sortent de l'intervalle -3 à 3.

Évidemment cette technique est longue et peu appropriée en présence d'un grand nombre d'observations. On peut donc identifier automatiquement les données extrêmes (*outliers*) en créant un objet qui contient toutes les observations dont les résidus excèdent l'intervalle de résidus acceptable. L'état de l'Alaska (st=2) ressort comme étant notre donnée extrême. 

In [ ]:
#Identification des résidus
rstandard(reglineaire)
outliers <- bd$st[rstandard(reglineaire) < -3 | rstandard(reglineaire) > 3]
outliers

On crée une nouvelle base de données excluant l'observation correspondante à la donnée extrême, selon l'une des deux méthodes suivantes.

In [ ]:
#Retrait de la donnée extrême
bd2 <- bd[-2, ]
bd2

In [ ]:
#Retrait des données extrêmes (lorsqu'il y en a plusieurs)
bd3 <- bd [-outliers,]
bd3

Maintenant que la données extrême est retirée, on refait simplement l'analyse de régression linéaire avec notre nouvelle base de données.

In [ ]:
#Refaire la régression linéaire sans les données extrêmes
reglineaire2 <- glm(jbs170 ~ scs134, data = bd2, family="gaussian")
summary(rstandard(reglineaire2))
summary(reglineaire2)
rsq(reglineaire2)

Interprétation:

Une fois la donnée extrême retirée, la régression linéaire démontre toujours l'existence d'une relation positive, de sorte que le taux de chômage augmente d’environ 0,23% pour chaque augmentation d’une unité du taux de décrochage scolaire. La relation est donc encore faible et positive. Le niveau de signification de la relation a augmenté (p < 0.001).

Le R-carré est passé à 0,317 , soit près du triple du r-carré avec données extrêmes. Ainsi, en dehors de l'Alaska, environ 32% de la variance du taux de chômage est expliqué par le taux de décrochage scolaire.